In [1]:
import os
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from trainer import Trainer
from model import ResidualUnet,DewarpingUNet
from loss import *
from DewarpingDataset import DewarpingDataset

In [2]:
# x = ["data/in/"+"_".join(i.split("_")[:-1])+".jpg" for i in os.listdir("data/out")]
# y = ["data/out/"+i for i in os.listdir("data/out")]
# paths = list(zip(y,x))
# pd.DataFrame(paths,columns=['generated','original']).to_csv("data.csv",index=False,header=False)

In [3]:
composed = transforms.Compose([transforms.Resize((512,512)),transforms.ToTensor()])

In [4]:
dataset = DewarpingDataset("data.csv",transform=composed)

In [5]:
train_index, test_index = train_test_split(np.arange(len(dataset)),random_state=0)

In [6]:
dataloader_training = DataLoader(dataset,sampler=train_index,batch_size=4)
dataloader_validation = DataLoader(dataset,sampler=train_index,batch_size=2)

In [7]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    torch.device('cpu')

# model
model = ResidualUnet(3,1).to(device)#DewarpingUNet(3,1).to(device)

# criterion
criterion =  MS_SSIM_Loss(data_range=1.0, size_average=True, channel=1)

# optimizer 
optimizer = torch.optim.SGD(model.parameters(), lr=10**(-4))

scheduler = torch.optim.lr_scheduler.StepLR(optimizer,  0.1, gamma=50)

In [8]:
# from lr_rate_finder import LearningRateFinder

# lrf = LearningRateFinder(model, criterion, optimizer, device)
# lrf.fit(dataloader_training, steps=1000)


In [9]:
# lrf.plot()

In [ ]:

# trainer
trainer = Trainer(model=model,
                  device=device,
                  criterion=criterion,
                  optimizer=optimizer,
                  training_DataLoader=dataloader_training,
                  validation_DataLoader=dataloader_validation,
                  lr_scheduler=scheduler,
                  epochs=20,
                  epoch=0,
                  notebook=True,
                  use_wandb=True,
                  save_model_name=f"model{model.__class__.__name__}_100epoches.pth")

# start training
training_losses, validation_losses, lr_rates = trainer.run_trainer()



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mfilan (use `wandb login --relogin` to force relogin)


Progress:   0%|          | 0/20 [00:00<?, ?it/s]

Training:   0%|          | 0/512 [00:00<?, ?it/s]

Validation:   0%|          | 0/1024 [00:00<?, ?it/s]

Training:   0%|          | 0/512 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/1024 [00:00<?, ?it/s]

Training:   0%|          | 0/512 [00:00<?, ?it/s]

Validation:   0%|          | 0/1024 [00:00<?, ?it/s]

Training:   0%|          | 0/512 [00:00<?, ?it/s]

Validation:   0%|          | 0/1024 [00:00<?, ?it/s]

In [ ]:
from tqdm import tqdm

In [13]:
len(test_index)

683

In [ ]:
batch_iter = tqdm(enumerate(dataloader_validation), 'Training', total=len(dataloader_validation),
                          leave=False)

for i, (x, y) in batch_iter:
    with torch.no_grad():
        input, target = x.to(device), y.to(device)  # send to device (GPU or CPU)
        out = model(input)  # one forward pass
    break

In [17]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    torch.device('cpu')

# model
model = DewarpingUNet(3,1).to(device)

# criterion
criterion =  DewarpingMS_SSIM_Loss(data_range=1.0, size_average=True, channel=1)

# optimizer 
optimizer = torch.optim.SGD(model.parameters(), lr=10**(-3.5))

scheduler = torch.optim.lr_scheduler.StepLR(optimizer,  0.1, gamma=50)

In [ ]:
from sklearn.externals._pilutil import bytescale

In [ ]:
def re_normalize(inp: np.ndarray, low: int = 0, high: int = 255):
    """Normalize the data to a certain range. Default: [0-255]"""
    inp_out = bytescale(inp, low=low, high=high)
    return inp_out

In [ ]:
plt.imshow(np.squeeze(target[0].detach().cpu().numpy()))

In [ ]:
Image.fromarray(re_normalize(np.moveaxis(np.squeeze(input[0].detach().cpu().numpy()),0, -1)))